In [204]:
import pandas as pd

In [205]:
xls_path = 'odir2019/ODIR-5K_Training_Annotations(Updated)_V2.xlsx'
df = pd.read_excel(xls_path)

In [206]:
df

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,4686,63,Male,4686_left.jpg,4686_right.jpg,severe nonproliferative retinopathy,proliferative diabetic retinopathy,0,1,0,0,0,0,0,0
3496,4688,42,Male,4688_left.jpg,4688_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
3497,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,0,0,0,0,0
3498,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0


In [207]:
# How many unique keywords are there?
#TODO:

In [208]:
# get unique diagnostic keywords
keywords = [ keyword  for keywords in df['Left-Diagnostic Keywords'] for keyword in keywords.split('，')]
unique_keywords = set(keywords)

print(keywords[:10])
print(unique_keywords)
print(len(unique_keywords),len(keywords))

['cataract', 'normal fundus', 'laser spot', 'moderate non proliferative retinopathy', 'normal fundus', 'macular epiretinal membrane', 'moderate non proliferative retinopathy', 'macular epiretinal membrane', 'drusen', 'normal fundus']
{'peripapillary atrophy', 'low image quality,maculopathy', 'suspected diabetic retinopathy', 'myelinated nerve fibers', 'suspected moderate non proliferative retinopathy', 'severe proliferative diabetic retinopathy', 'fundus laser photocoagulation spots', 'retinochoroidal coloboma', 'retinal pigment epithelium atrophy', 'myopia retinopathy', 'hypertensive retinopathy,diabetic retinopathy', 'arteriosclerosis', 'optic disk photographically invisible', 'maculopathy', 'retinal pigmentation', 'wet age-related macular degeneration', 'optic disk epiretinal membrane', 'drusen', 'vessel tortuosity', 'pathological myopia', 'wedge white line change', 'asteroid hyalosis', 'suspected glaucoma', 'atrophy', 'macular epiretinal membrane', 'retinitis pigmentosa', 'severe n

In [209]:
# create a mapping from keywords to class labels
class_labels = ['N','D','G','C','A','H','M','O']
keyword_label_mapping  = {
    'normal':'N',
    'retinopathy':'D',
    'glaucoma':'G',
    'cataract':'C',
    'macular degeneration':'A',
    'hypertensive':'H',
    'myopia':'M',
}
non_decisive_labels = ["lens dust", "optic disk photographically invisible", "low image quality", "image offset"]
# if the keyword contains label outside of the above then, label them as others 'O'

In [210]:
def generate_individual_label(diagnostic_keywords):
    keywords = [ keyword  for keyword in diagnostic_keywords.split('，')]
    contains_normal = False
    for k in keywords:
        for label in keyword_label_mapping.keys():
            if label in k:
                if label == 'normal':
                    contains_normal = True # if found a 'normal' keyword, check if there are other keywords but keep in mind that a normal keyword was found
                else:
                    return keyword_label_mapping[label] # found a proper keyword label, use the first occurence

    # did not find a proper keyword label, see if there are labels other than non-decisive labels, if so, categorize them as 'others'
    decisive_label = False
    for k in keywords:
        if k not in non_decisive_labels and (('normal' not in k) or ('abnormal' in k)):
            decisive_label = True
    if decisive_label:
        # contains decisive label other than the normal and abnormal categories
        return 'O' 
    if contains_normal:
        return 'N'
    
    # if any of the above criteria do not match, then return as is
    return keywords # useful for diagnostics, check if there are cases that are not covered by the above


In [211]:
generate_individual_label('normal fundus')
generate_individual_label('lens dust，drusen，normal fundus	')

'O'

In [212]:
df['Left-label']= df['Left-Diagnostic Keywords'].apply(generate_individual_label)
df['Right-label'] = df['Right-Diagnostic Keywords'].apply(generate_individual_label)

In [213]:
df.to_csv('ODIR-5K_Training_Annotations_processed.csv')

In [214]:
# write test cases
# if both left and right are normal, then the final diagnosis is also normal
def test_normal(row):
    l,r = row['Left-label'], row['Right-label']
    if l == 'N' and r == 'N' and row['N'] != 1:
        return False
    else:
        return True

def test_others(row):
    l,r = row['Left-label'], row['Right-label']
    if row['O'] == 1:
        if l == 'O' or r == 'O':
            return True
        else:
            return False 
    return True

# find rows where both left and right have beeen processed as Normal, but the final diagnosis is not 'N
df[df.apply(test_normal, axis=1) == False]
# find rows where none of the left and right have been processed as Others, but the final diagnosis also contains 'O'
df[df.apply(test_others,axis=1) == False]



,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,Left-label,Right-label
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,D,D
22,22,55,Female,22_left.jpg,22_right.jpg,moderate non proliferative retinopathy，laser spot,laser spot，moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,D,D
26,26,63,Female,26_left.jpg,26_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy，abnorma...,0,1,0,0,0,0,0,1,D,D
27,27,33,Male,27_left.jpg,27_right.jpg,normal fundus,macular epiretinal membrane，moderate non proli...,0,1,0,0,0,0,0,1,N,D
37,37,41,Male,37_left.jpg,37_right.jpg,macular epiretinal membrane，mild nonproliferat...,normal fundus,0,1,0,0,0,0,0,1,D,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3281,4435,67,Male,4435_left.jpg,4435_right.jpg,post retinal laser surgery，moderate non prolif...,post retinal laser surgery，proliferative diabe...,0,1,0,1,0,0,0,1,D,D
3325,4485,49,Female,4485_left.jpg,4485_right.jpg,post retinal laser surgery，moderate non prolif...,post retinal laser surgery，moderate non prolif...,0,1,0,0,0,0,0,1,D,D
3340,4501,62,Female,4501_left.jpg,4501_right.jpg,post retinal laser surgery，proliferative diabe...,post retinal laser surgery，proliferative diabe...,0,1,0,0,0,0,0,1,D,D
3426,4598,64,Male,4598_left.jpg,4598_right.jpg,post retinal laser surgery，severe nonprolifera...,post retinal laser surgery，severe nonprolifera...,0,1,0,0,0,0,0,1,D,D


In [216]:
manually_remove = ['2174_right.jpg',
'2175_left.jpg',
'2176_left.jpg',
'2177_left.jpg',
'2177_right.jpg',
'2178_right.jpg',
'2179_left.jpg',
'2179_right.jpg',
'2180_left.jpg',
'2180_right.jpg',
'2181_left.jpg',
'2181_right.jpg',
'2182_left.jpg',
'2182_right.jpg',
'2957_left.jpg',
'2957_right.jpg',
]

df = df[(~df['Left-Fundus'].isin(manually_remove)) & (~df['Right-Fundus'].isin(manually_remove))]

# create a new dataframe where each row corresponds to one image
left_fundus = df['Left-Fundus']
left_label = df['Left-label']
left_keywords = df['Left-Diagnostic Keywords']
right_fundus = df['Right-Fundus']
right_label = df['Right-label']
right_keywords = df['Right-Diagnostic Keywords']
id = df['ID']
age = df['Patient Age']
sex = df['Patient Sex']

# separate train and test split
from sklearn.model_selection import train_test_split
SEED = 234
id_train, id_val = train_test_split(id,test_size=0.1,random_state=SEED)

train_left_fundus = df[df['ID'].isin(id_train)]['Left-Fundus']
train_left_label = df[df['ID'].isin(id_train)]['Left-label']
train_left_keywords = df[df['ID'].isin(id_train)]['Left-Diagnostic Keywords']

train_right_fundus = df[df['ID'].isin(id_train)]['Right-Fundus']
train_right_label = df[df['ID'].isin(id_train)]['Right-label']
train_right_keywords = df[df['ID'].isin(id_train)]['Right-Diagnostic Keywords']


val_left_fundus = df[df['ID'].isin(id_val)]['Left-Fundus']
val_left_label = df[df['ID'].isin(id_val)]['Left-label']
val_left_keywords = df[df['ID'].isin(id_val)]['Left-Diagnostic Keywords']

val_right_fundus = df[df['ID'].isin(id_val)]['Right-Fundus']
val_right_label = df[df['ID'].isin(id_val)]['Right-label']
val_right_keywords = df[df['ID'].isin(id_val)]['Right-Diagnostic Keywords']

# stack left and right columns vertically
train_fundus = pd.concat([train_left_fundus, train_right_fundus],axis=0,ignore_index=True,sort=True)
train_label = pd.concat([train_left_label,  train_right_label],axis=0,ignore_index=True,sort=True)
train_keywords = pd.concat([train_left_keywords,train_right_keywords],axis=0,ignore_index=True,sort=True)

val_fundus = pd.concat([val_left_fundus, val_right_fundus],axis=0,ignore_index=True)
val_label = pd.concat([val_left_label,val_right_label],axis=0,ignore_index=True)
val_keywords = pd.concat([val_left_keywords,val_right_keywords],axis=0,ignore_index=True)

train_df_left_right_separate_row = pd.concat([train_fundus,
                                              train_label,
                                              train_keywords],axis=1,sort=True,
                                              keys = ['fundus','label','keywords']) # stack horizontally
val_df_left_right_separate_row = pd.concat([  val_fundus,
                                              val_label,
                                              val_keywords],axis=1,sort=True,
                                              keys=['fundus','label','keywords']) # stack horizontally

In [217]:
train_df_left_right_separate_row.to_csv('processed_train_ODIR-5K.csv')
val_df_left_right_separate_row.to_csv('processed_val_ODIR-5K.csv')

In [218]:
len(df),len(id_train),len(id_val),len(train_fundus),len(val_fundus),len(train_df_left_right_separate_row),len(val_df_left_right_separate_row)

(3490, 3141, 349, 6282, 698, 6282, 698)